In [9]:
import torch
import torch.nn as nn
import os
import cv2
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [10]:
from torchvision import models
from torchvision import transforms
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [11]:
if(os.path.split(os.getcwd())[1] == "misc"):
    os.chdir("..")
print("Current Working Directory: {}".format(os.path.split(os.getcwd())[1]))

cuda = True

Current Working Directory: progettoVIPM


In [12]:
from utils.loadersAndEnums import datasets
from utils.loadersAndEnums import networks
from utils.loadersAndEnums import ImageDataset

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# VALIDATION SET

## BASELINE

In [18]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2), # 256x256x3 -> 256x256x32
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)) # 256x256x32 -> 128x128x32   
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2), # 128x128x32 -> 128x128x64
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)) # 128x128x64 -> 64x64x64
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2), # 64x64x64 -> 64x64x128
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)) # 64x64x128 -> 32x32x128
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(32*32*128, 500)
        self.fc2 = nn.Linear(500, 251)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = nn.ReLU()(out)
        out = self.fc2(out)
        return out
    
model=ConvNet()
model.load_state_dict(torch.load('misc/models/best_baseline.pth'))
from torch.utils.data import DataLoader
val = ImageDataset(datasets.VALIDATION_LABELED_20,256,True)
val = DataLoader(val,batch_size=1,shuffle=False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
from utils.fine_tune_pytorch import eval_model_on_test_loader

cm = eval_model_on_test_loader(model, "Baseline_Validation", "misc", val, True, 5)

100%|██████████| 1004/1004 [00:06<00:00, 152.66it/s]

Accuracy: 1.593625498007968 %
5-Accuracy: 6.175298804780876


## SPARSE AUTOENCODER

In [19]:
from utils.sparseAutoencoder import SparseAutoencoderL1
best_model_path = 'self-supervised/models/best_sparse_AE.pth'
sparseAE = SparseAutoencoderL1()
sparseAE.load_state_dict(torch.load(best_model_path))
sparseAE.eval()  # Set the model to evaluation mode
sparseAE.to(device)
class ExtendedEncoderV2(nn.Module):
    def __init__(self, base_encoder):
        super(ExtendedEncoderV2, self).__init__()
        self.base_encoder = base_encoder

        # Additional convolutional and ReLU layers
        self.additional_conv = nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1)
        self.additional_relu = nn.ReLU()

        # Dynamically compute flatten_dim
        dummy_input = torch.randn(1, 3, 256, 256).to(device)  # Simulate input
        self.additional_conv.to(device)  # Move additional_conv to the same device
        dummy_output = self.base_encoder(dummy_input)  # Get encoder output
        dummy_output = self.additional_conv(dummy_output)  # Pass through additional conv layer
        dummy_output = self.additional_relu(dummy_output)  # Pass through additional ReLU layer
        self.flatten_dim = dummy_output.view(1, -1).size(1)  # Flatten dimension

        # Fully connected layers with dropout
        self.fc1 = nn.Linear(self.flatten_dim, 260)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(260, 251)
        self.dropout2 = nn.Dropout(p=0.5)
        self.softmax = nn.Softmax(dim=1)

        # Freeze base encoder weights
        for param in self.base_encoder.parameters():
            param.requires_grad = False

    def forward(self, x):
        x = self.base_encoder(x)  # Base encoder
        x = self.additional_conv(x)  # Additional conv layer
        x = self.additional_relu(x)  # Additional ReLU layer
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc1(x)  # Fully connected layer
        x = self.relu(x)  # ReLU activation
        x = self.dropout1(x)  # Dropout
        x = self.fc2(x)  # Fully connected layer
        x = self.dropout2(x)  # Dropout
        x = self.softmax(x)  # Softmax activation
        return x
    
    
from torch.utils.data import DataLoader
val = ImageDataset(datasets.VALIDATION_LABELED_20,256,True)
val = DataLoader(val,batch_size=1,shuffle=False)
exEncoder = ExtendedEncoderV2(sparseAE.encoder)
exEncoder.load_state_dict(torch.load("self-supervised/models/best_AE_classifier.pth"))
exEncoder.to(device)
from utils.fine_tune_pytorch import eval_model_on_test_loader

cm = eval_model_on_test_loader(exEncoder, "sparse_encoder_classifier_validation", "misc", val, True, 5)

  0%|          | 0/1004 [00:00<?, ?it/s]

100%|██████████| 1004/1004 [00:04<00:00, 226.31it/s]

Accuracy: 0.796812749003984 %
5-Accuracy: 2.091633466135458


## COLORIZER

In [20]:
from sklearn.metrics import accuracy_score


class ColorizationModel(nn.Module):
    def __init__(self):
        super(ColorizationModel, self).__init__()
        # Encoder: Convolutional layers
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output size: 64 x 128 x 128

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # Output size: 128 x 64 x 64

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) # Output size: 256 x 32 x 32
        )
        # Decoder: Deconvolutional layers
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.ConvTranspose2d(64, 2, kernel_size=4, stride=2, padding=1),  # Output 2 channels (a, b)
            nn.Tanh()  # Normalize output to range [-1, 1]
        )

    def forward(self, x):
        features = self.encoder(x)
        output = self.decoder(features)
        return output
    
best_model_path = 'self-supervised/models/best_Colorizer.pth'

colorizer = ColorizationModel()
colorizer.load_state_dict(torch.load(best_model_path))
colorizer.to(device)
class ExtendedColorizerLinear(nn.Module):
    def __init__(self, colorizer):
        super(ExtendedColorizerLinear, self).__init__()
        self.encoder = colorizer.encoder
        self.ff1 = nn.Linear(256*32*32, 1024)
        self.relu = nn.ReLU()
        self.ff2 = nn.Linear(1024, 251)
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(p=0.5)
        for param in self.encoder.parameters():
            param.requires_grad = False
    def forward(self, x):
        x = self.encoder(x)
        x = x.view(-1, 256*32*32)
        x = self.ff1(x)
        x = self.dropout(x)
        x = self.softmax(x)
        return x
    
exColorizer = ExtendedColorizerLinear(colorizer)

best_model_path = 'self-supervised/models/best_LinearColorizer_classifier.pth'

exColorizer.load_state_dict(torch.load(best_model_path))
exColorizer.to(device)
def process_batch(batch):
    batch = batch.cpu().numpy()
    batch = np.moveaxis(batch, 1, -1)  # Move channel axis to the end
    lab_batch = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2LAB) for img in batch])
    
    L_batch = lab_batch[:,:,:,0] / 255.0
    ab_batch = lab_batch[:,:,:,1:] / 128.0
    
    L_batch = torch.tensor(L_batch, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
    ab_batch = torch.tensor(ab_batch, dtype=torch.float32).permute(0, 3, 1, 2)  # Move channel axis to the second position
    
    return L_batch, ab_batch

k = 5
k_correct = 0
total = len(val)
exColorizer.eval()
predictions = np.zeros(len(val))
# Disable gradient computation and reduce memory consumption.
y_test = np.zeros(len(val))
with torch.no_grad():
    i = 0
    for test_data in tqdm(val):
        test_features, test_labels = test_data
        test_features,_ = process_batch(test_features)
        test_features = test_features.to(device)
        y_test[i] = test_labels.cpu()
        outputs = exColorizer(test_features)
        _, predicted = torch.max(outputs, 1)
        predictions[i] = predicted.item()
        _, indexes = torch.sort(outputs, descending=True)
        indexes = indexes[0,0:k]
        if y_test[i] in indexes:
            k_correct += 1
        i+=1

k_accuracy = 100*(k_correct/total)
accuracy = 100*accuracy_score(y_test, predictions)
print("Accuracy: {}".format(accuracy))
print("{}-Accuracy: {}".format(k, k_accuracy))
cm = confusion_matrix(y_test, predictions)
cm = np.array(cm)
np.save("./{}/model_metrics/ConfM_{}.npy".format("misc", "Linear_Colorizer_Classifier_validation"), cm)

100%|██████████| 1004/1004 [00:13<00:00, 73.22it/s]

Accuracy: 1.0956175298804782
5-Accuracy: 3.386454183266932


## COLORIZER + DATA AUGMENTATION

In [21]:
from sklearn.metrics import accuracy_score


class ColorizationModel(nn.Module):
    def __init__(self):
        super(ColorizationModel, self).__init__()
        # Encoder: Convolutional layers
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output size: 64 x 128 x 128

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # Output size: 128 x 64 x 64

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) # Output size: 256 x 32 x 32
        )
        # Decoder: Deconvolutional layers
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.ConvTranspose2d(64, 2, kernel_size=4, stride=2, padding=1),  # Output 2 channels (a, b)
            nn.Tanh()  # Normalize output to range [-1, 1]
        )

    def forward(self, x):
        features = self.encoder(x)
        output = self.decoder(features)
        return output
    
best_model_path = 'self-supervised/models/best_Colorizer.pth'

colorizer = ColorizationModel()
colorizer.load_state_dict(torch.load(best_model_path))
colorizer.to(device)
class ExtendedColorizerLinear(nn.Module):
    def __init__(self, colorizer):
        super(ExtendedColorizerLinear, self).__init__()
        self.encoder = colorizer.encoder
        self.ff1 = nn.Linear(256*32*32, 1024)
        self.relu = nn.ReLU()
        self.ff2 = nn.Linear(1024, 251)
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(p=0.5)
        for param in self.encoder.parameters():
            param.requires_grad = False
    def forward(self, x):
        x = self.encoder(x)
        x = x.view(-1, 256*32*32)
        x = self.ff1(x)
        x = self.dropout(x)
        x = self.softmax(x)
        return x
    
exColorizer = ExtendedColorizerLinear(colorizer)

best_model_path = 'self-supervised/models/best_Augmented_LinearColorizer_classifier.pth'

exColorizer.load_state_dict(torch.load(best_model_path))
exColorizer.to(device)
def process_batch(batch):
    batch = batch.cpu().numpy()
    batch = np.moveaxis(batch, 1, -1)  # Move channel axis to the end
    lab_batch = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2LAB) for img in batch])
    
    L_batch = lab_batch[:,:,:,0] / 255.0
    ab_batch = lab_batch[:,:,:,1:] / 128.0
    
    L_batch = torch.tensor(L_batch, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
    ab_batch = torch.tensor(ab_batch, dtype=torch.float32).permute(0, 3, 1, 2)  # Move channel axis to the second position
    
    return L_batch, ab_batch

k = 5
k_correct = 0
total = len(val)
exColorizer.eval()
predictions = np.zeros(len(val))
# Disable gradient computation and reduce memory consumption.
y_test = np.zeros(len(val))
with torch.no_grad():
    i = 0
    for test_data in tqdm(val):
        test_features, test_labels = test_data
        test_features,_ = process_batch(test_features)
        test_features = test_features.to(device)
        y_test[i] = test_labels.cpu()
        outputs = exColorizer(test_features)
        _, predicted = torch.max(outputs, 1)
        predictions[i] = predicted.item()
        _, indexes = torch.sort(outputs, descending=True)
        indexes = indexes[0,0:k]
        if y_test[i] in indexes:
            k_correct += 1
        i+=1

k_accuracy = 100*(k_correct/total)
accuracy = 100*accuracy_score(y_test, predictions)
print("Accuracy: {}".format(accuracy))
print("{}-Accuracy: {}".format(k, k_accuracy))
cm = confusion_matrix(y_test, predictions)
cm = np.array(cm)
np.save("./{}/model_metrics/ConfM_{}.npy".format("misc", "Augmented_Linear_Colorizer_Classifier_validation"), cm)

100%|██████████| 1004/1004 [00:14<00:00, 71.00it/s]

Accuracy: 0.49800796812749004
5-Accuracy: 2.788844621513944


## COLORIZER + LABELS CLUSTERING

In [22]:
from sklearn.metrics import accuracy_score


class ColorizationModel(nn.Module):
    def __init__(self):
        super(ColorizationModel, self).__init__()
        # Encoder: Convolutional layers
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output size: 64 x 128 x 128

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # Output size: 128 x 64 x 64

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) # Output size: 256 x 32 x 32
        )
        # Decoder: Deconvolutional layers
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.ConvTranspose2d(64, 2, kernel_size=4, stride=2, padding=1),  # Output 2 channels (a, b)
            nn.Tanh()  # Normalize output to range [-1, 1]
        )

    def forward(self, x):
        features = self.encoder(x)
        output = self.decoder(features)
        return output
    
best_model_path = 'self-supervised/models/best_Colorizer.pth'

colorizer = ColorizationModel()
colorizer.load_state_dict(torch.load(best_model_path))
colorizer.to(device)
class ExtendedColorizerLinear(nn.Module):
    def __init__(self, colorizer):
        super(ExtendedColorizerLinear, self).__init__()
        self.encoder = colorizer.encoder
        self.ff1 = nn.Linear(256*32*32, 1024)
        self.relu = nn.ReLU()
        self.ff2 = nn.Linear(1024, 251)
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(p=0.5)
        for param in self.encoder.parameters():
            param.requires_grad = False
    def forward(self, x):
        x = self.encoder(x)
        x = x.view(-1, 256*32*32)
        x = self.ff1(x)
        x = self.dropout(x)
        x = self.softmax(x)
        return x
    
exColorizer = ExtendedColorizerLinear(colorizer)

best_model_path = 'self-supervised/models/best_Label_Extended_LinearColorizer_classifier.pth'

exColorizer.load_state_dict(torch.load(best_model_path))
exColorizer.to(device)
def process_batch(batch):
    batch = batch.cpu().numpy()
    batch = np.moveaxis(batch, 1, -1)  # Move channel axis to the end
    lab_batch = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2LAB) for img in batch])
    
    L_batch = lab_batch[:,:,:,0] / 255.0
    ab_batch = lab_batch[:,:,:,1:] / 128.0
    
    L_batch = torch.tensor(L_batch, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
    ab_batch = torch.tensor(ab_batch, dtype=torch.float32).permute(0, 3, 1, 2)  # Move channel axis to the second position
    
    return L_batch, ab_batch

k = 5
k_correct = 0
total = len(val)
exColorizer.eval()
predictions = np.zeros(len(val))
# Disable gradient computation and reduce memory consumption.
y_test = np.zeros(len(val))
with torch.no_grad():
    i = 0
    for test_data in tqdm(val):
        test_features, test_labels = test_data
        test_features,_ = process_batch(test_features)
        test_features = test_features.to(device)
        y_test[i] = test_labels.cpu()
        outputs = exColorizer(test_features)
        _, predicted = torch.max(outputs, 1)
        predictions[i] = predicted.item()
        _, indexes = torch.sort(outputs, descending=True)
        indexes = indexes[0,0:k]
        if y_test[i] in indexes:
            k_correct += 1
        i+=1

k_accuracy = 100*(k_correct/total)
accuracy = 100*accuracy_score(y_test, predictions)
print("Accuracy: {}".format(accuracy))
print("{}-Accuracy: {}".format(k, k_accuracy))
cm = confusion_matrix(y_test, predictions)
cm = np.array(cm)
np.save("./{}/model_metrics/ConfM_{}.npy".format("misc", "Label_Extended_Linear_Colorizer_Classifier_validation"), cm)

100%|██████████| 1004/1004 [00:13<00:00, 72.31it/s]

Accuracy: 0.398406374501992
5-Accuracy: 2.49003984063745


## PRETRAINED RESNET 50 (FOOD101)

In [26]:
from torch.utils.data import DataLoader

val = ImageDataset(datasets.VALIDATION_LABELED_20,224,True)
val = DataLoader(val,batch_size=1,shuffle=False)
model = models.resnet50()
num_classes = 251
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("Transfer_Learning/models/best_FineTuned_ResNet50.pth"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
from utils.fine_tune_pytorch import eval_model_on_test_loader

cm = eval_model_on_test_loader(model, "Finetuned_FOOD101_ResNet50_validation", "misc", val, True, 5)

100%|██████████| 1004/1004 [00:11<00:00, 90.99it/s]

Accuracy: 32.669322709163346 %
5-Accuracy: 59.26294820717132


## PRETRAINED RESNET 50 (IMAGENET1K)

In [27]:
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
num_classes = 251
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("Transfer_Learning/models/best_FineTuned_ResNet50_NO_FREEZE.pth"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
from torch.utils.data import DataLoader
val = ImageDataset(datasets.VALIDATION_LABELED_20,224,True)
val = DataLoader(val,batch_size=1,shuffle=False)
from utils.fine_tune_pytorch import eval_model_on_test_loader
cm = eval_model_on_test_loader(model, "Finetuned_IMAGENET1K_ResNet50_validation", "misc", val, True, 5)

100%|██████████| 1004/1004 [00:10<00:00, 92.95it/s]

Accuracy: 25.39840637450199 %
5-Accuracy: 49.40239043824701


## CLASSIFY TO LEARN RESNET 50 (FOOD101)

In [23]:

from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import numpy as np
data = np.fromfile('misc/models/all_out_from_resnet_5080.dat')

val = ImageDataset(datasets.VALIDATION_LABELED_20,224,True)
val = DataLoader(val,batch_size=1,shuffle=False)
k = 5
k_correct = 0
total = len(val)
predictions = np.zeros(len(val))
# Disable gradient computation and reduce memory consumption.
y_test = np.zeros(len(val))

i = 0
for test_data in tqdm(val):
    test_features, test_labels = test_data
    y_test[i] = test_labels.cpu()
    outputs = torch.tensor(data[i*251:i*251+251], device=device).unsqueeze(0)
    _, predicted = torch.max(outputs, 1)
    predictions[i] = predicted.item()
    _, indexes = torch.sort(outputs, descending=True)
    indexes = indexes[0, :k]
    if y_test[i] in indexes:
        k_correct += 1
    i+=1

k_accuracy = 100*(k_correct/total)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: {} %".format(accuracy *100))
print("{}-Accuracy: {}".format(k, k_accuracy))
cm = confusion_matrix(y_test, predictions)
cm = np.array(cm)
np.save("./{}/model_metrics/ConfM_{}.npy".format("misc", "classify_to_learn_extended_Finetuned_ResNet50_validation"), cm)

100%|██████████| 1004/1004 [00:03<00:00, 278.03it/s]

Accuracy: 0.29880478087649404 %
5-Accuracy: 1.693227091633466


In [20]:
print(data[i:i+251].shape)

(251,)


## PRETRAINED RESNET50(IMAGENET1K) AUGMENTED TRAINING

### Pytorch Weights 

In [29]:
from torch.utils.data import DataLoader
val = ImageDataset(datasets.VALIDATION_20,224,True)
val = DataLoader(val,batch_size=1,shuffle=False)
model = models.resnet50()
num_classes = 251
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("Transfer_Learning/models/best_FineTuned_ResNet50_NO_FREEZE_DEGRADED_DATA_HALF_LR.pth"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
from utils.fine_tune_pytorch import eval_model_on_test_loader
print("VALIDATION:")
cm = eval_model_on_test_loader(model, "Finetuned_IMAGENET1K_ResNet50_Data_Augmentation_PyTorch_Weights_validation_", "misc", val, True, 5)
valD = ImageDataset(datasets.VALIDATION_20_DEGRADED,224,True)
valD = DataLoader(valD,batch_size=1,shuffle=False)
print("DEGRADED VALIDATION:")
cm = eval_model_on_test_loader(model, "Finetuned_IMAGENET1K_ResNet50_Data_Augmentation_PyTorch_Weights_validation_degraded", "misc", valD, True, 5)

VALIDATION:


100%|██████████| 1004/1004 [00:10<00:00, 94.15it/s]


Accuracy: 22.808764940239044 %
5-Accuracy: 46.31474103585657
DEGRADED VALIDATION:


100%|██████████| 1004/1004 [00:11<00:00, 90.93it/s]

Accuracy: 16.235059760956176 %
5-Accuracy: 35.55776892430279


### Trained Weights

In [24]:
from torch.utils.data import DataLoader
val = ImageDataset(datasets.VALIDATION_20,224,True)
val = DataLoader(val,batch_size=1,shuffle=False)
model = models.resnet50()
num_classes = 251
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("Transfer_Learning/models/best_FineTuned_ResNet50_NO_FREEZE_DEGRADED_DATA_ALREADY_FINETUNED.pth"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
from utils.fine_tune_pytorch import eval_model_on_test_loader
print("VALIDATION:")
cm = eval_model_on_test_loader(model, "Finetuned_IMAGENET1K_ResNet50_Data_Augmentation_Finetuned_Weights_validation_", "misc", val, True, 5)
valD = ImageDataset(datasets.VALIDATION_20_DEGRADED,224,True)
valD = DataLoader(valD,batch_size=1,shuffle=False)
print("DEGRADED VALIDATION:")
cm = eval_model_on_test_loader(model, "Finetuned_IMAGENET1K_ResNet50_Data_Augmentation_Finetuned_Weights_validation_degraded", "misc", valD, True, 5)

VALIDATION:


100%|██████████| 1004/1004 [00:11<00:00, 88.64it/s]


Accuracy: 26.195219123505975 %
5-Accuracy: 52.788844621513945
DEGRADED VALIDATION:


100%|██████████| 1004/1004 [00:11<00:00, 86.17it/s]

Accuracy: 18.92430278884462 %
5-Accuracy: 36.05577689243028


## PRETRAINED RESNET50 (FOOD101) AUGMENTED TRAINING

### ORIGINAL WEIGHTS

In [27]:
from torch.utils.data import DataLoader
val = ImageDataset(datasets.VALIDATION_20,224,True)
val = DataLoader(val,batch_size=1,shuffle=False)
model = models.resnet50()
num_classes = 251
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = torch.load("Transfer_Learning/models/finetunedResNet50_minusnessuno_10e_64bsize_80_20_split_dataset_degraded_train_whole.pth")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
from utils.fine_tune_pytorch import eval_model_on_test_loader
print("VALIDATION:")
cm = eval_model_on_test_loader(model, "Finetuned_FOOD101_ResNet50_Data_Augmentation_Origina_Weights_validation_", "misc", val, True, 5)
valD = ImageDataset(datasets.VALIDATION_20_DEGRADED,224,True)
valD = DataLoader(valD,batch_size=1,shuffle=False)
print("DEGRADED VALIDATION:")
cm = eval_model_on_test_loader(model, "Finetuned_FOOD101_ResNet50_Data_Augmentation_Origina_Weights_validation_degraded", "misc", valD, True, 5)

VALIDATION:


100%|██████████| 1004/1004 [00:11<00:00, 88.03it/s]


Accuracy: 28.984063745019924 %
5-Accuracy: 54.482071713147405
DEGRADED VALIDATION:


100%|██████████| 1004/1004 [00:11<00:00, 89.56it/s]


Accuracy: 24.40239043824701 %
5-Accuracy: 47.808764940239044


### FINETUNED WEIGHTS

In [28]:
from torch.utils.data import DataLoader
val = ImageDataset(datasets.VALIDATION_20,224,True)
val = DataLoader(val,batch_size=1,shuffle=False)
model = models.resnet50()
num_classes = 251
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = torch.load("Transfer_Learning/models/10e_64bsize_80_20_split_dataset_degraded_from_previous_weights.pth")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
from utils.fine_tune_pytorch import eval_model_on_test_loader
print("VALIDATION:")
cm = eval_model_on_test_loader(model, "Finetuned_FOOD101_ResNet50_Data_Augmentation_Finetuned_Weights_validation_", "misc", val, True, 5)
valD = ImageDataset(datasets.VALIDATION_20_DEGRADED,224,True)
valD = DataLoader(valD,batch_size=1,shuffle=False)
print("DEGRADED VALIDATION:")
cm = eval_model_on_test_loader(model, "Finetuned_FOOD101_ResNet50_Data_Augmentation_Finetuned_Weights_validation_degraded", "misc", valD, True, 5)

VALIDATION:


100%|██████████| 1004/1004 [00:11<00:00, 86.60it/s]


Accuracy: 28.286852589641438 %
5-Accuracy: 51.79282868525896
DEGRADED VALIDATION:


100%|██████████| 1004/1004 [00:11<00:00, 86.86it/s]

Accuracy: 23.107569721115535 %
5-Accuracy: 45.41832669322709


# TEST SETS

## PRETRAINED RESNET50 (FOOD101)

N.B.: Results are in order: TEST, TEST_DEGRADED

In [29]:
from torch.utils.data import DataLoader
test = ImageDataset(datasets.TEST,224,True)
test = DataLoader(test,batch_size=1,shuffle=False)
model = models.resnet50()
num_classes = 251
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("Transfer_Learning/models/best_FineTuned_ResNet50.pth"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
from utils.fine_tune_pytorch import eval_model_on_test_loader

cm = eval_model_on_test_loader(model, "finetuned_pretrained_ResNet50_Food101_classifier_test", "misc", test, True, 5)
testD = ImageDataset(datasets.TEST_DEGRADED,224,True)
testD = DataLoader(testD,batch_size=1,shuffle=False)
cm = eval_model_on_test_loader(model, "finetuned_pretrained_ResNet50_Food101_classifier_test_degraded", "misc", testD, True, 5)

100%|██████████| 11994/11994 [02:30<00:00, 79.68it/s]


Accuracy: 39.73653493413373 %
5-Accuracy: 67.60046690011673


100%|██████████| 11994/11994 [02:23<00:00, 83.75it/s]

Accuracy: 26.721694180423544 %
5-Accuracy: 48.79939969984993


## PRETRAINED RESNET50 (IMAGENET1K)

In [30]:
from torch.utils.data import DataLoader
test = ImageDataset(datasets.TEST,224,True)
test = DataLoader(test,batch_size=1,shuffle=False)
model = models.resnet50()
num_classes = 251
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("Transfer_Learning/models/best_FineTuned_ResNet50_NO_FREEZE.pth"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
from utils.fine_tune_pytorch import eval_model_on_test_loader

cm = eval_model_on_test_loader(model, "finetuned_pretrained_ResNet50_ImageNet1k_classifier_test", "misc", test, True, 5)
testD = ImageDataset(datasets.TEST_DEGRADED,224,True)
testD = DataLoader(testD,batch_size=1,shuffle=False)
cm = eval_model_on_test_loader(model, "finetuned_pretrained_ResNet50_ImageNet1k_classifier_test_degraded", "misc", testD, True, 5)

100%|██████████| 11994/11994 [02:25<00:00, 82.51it/s]


Accuracy: 29.87326996831749 %
5-Accuracy: 55.61113890278473


100%|██████████| 11994/11994 [02:16<00:00, 87.70it/s]

Accuracy: 18.325829581457395 %
5-Accuracy: 36.87677171919293


## PRETRAINED RESNET50(FOOD101) AUGMENTED TRAINING

In [31]:
from torch.utils.data import DataLoader
test = ImageDataset(datasets.TEST,224,True)
test = DataLoader(test,batch_size=1,shuffle=False)
model = models.resnet50()
num_classes = 251
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = torch.load("Transfer_Learning/models/finetunedResNet50_minusnessuno_10e_64bsize_80_20_split_dataset_degraded_train_whole.pth")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
from utils.fine_tune_pytorch import eval_model_on_test_loader

cm = eval_model_on_test_loader(model, "Finetuned_FOOD101_ResNet50_Data_Augmentation_Origina_Weights_test_", "misc", test, True, 5)
testD = ImageDataset(datasets.TEST_DEGRADED,224,True)
testD = DataLoader(testD,batch_size=1,shuffle=False)
cm = eval_model_on_test_loader(model, "Finetuned_FOOD101_ResNet50_Data_Augmentation_Origina_Weights_test_degraded", "misc", testD, True, 5)

100%|██████████| 11994/11994 [02:18<00:00, 86.86it/s]


Accuracy: 35.05919626479906 %
5-Accuracy: 62.673003168250794


100%|██████████| 11994/11994 [02:23<00:00, 83.49it/s]

Accuracy: 29.214607303651825 %
5-Accuracy: 54.98582624645656


In [1]:
accuracy_validation = {}
top5_accuracy_validation = {}
accuracy_test = {}
top5_accuracy_test = {}
accuracy_validation_degraded = {}
top5_accuracy_validation_degraded = {}
accuracy_test_degraded = {}
top5_accuracy_test_degraded = {}
accuracy_validation["Baseline"] = 1.593625498007968
top5_accuracy_validation["Baseline"] = 6.175298804780876
accuracy_validation["Sparse Autoencoder"] = 0.796812749003984
top5_accuracy_validation["Sparse Autoencoder"] = 2.091633466135458
accuracy_validation["Colorizer"] = 1.0956175298804782
top5_accuracy_validation["Colorizer"] = 3.386454183266932
accuracy_validation["Colorizer + Data Augmentation"] = 0.49800796812749004
top5_accuracy_validation["Colorizer + Data Augmentation"] = 2.788844621513944
accuracy_validation["Colorizer + Labels Clustering"] = 0.398406374501992
top5_accuracy_validation["Colorizer + Labels Clustering"] = 2.49003984063745
accuracy_validation["Pretrained ResNet50 (FOOD101)"] = 32.669322709163346
top5_accuracy_validation["Pretrained ResNet50 (FOOD101)"] = 59.26294820717132
accuracy_validation["Pretrained ResNet50 (ImageNet1k)"] = 25.39840637450199
top5_accuracy_validation["Pretrained ResNet50 (ImageNet1k)"] = 49.40239043824701
accuracy_validation["Classify To Learn ResNet50"] = 0.29880478087649404
top5_accuracy_validation["Classify To Learn ResNet50"] = 1.693227091633466
accuracy_validation["Pretrained ResNet50 (ImageNet1k) Augmented Training Pytorch Weights"] = 22.808764940239044
top5_accuracy_validation["Pretrained ResNet50 (ImageNet1k) Augmented Training Pytorch Weights"] =  46.31474103585657
accuracy_validation_degraded["Pretrained ResNet50 (ImageNet1k) Augmented Training Pytorch Weights"] = 16.235059760956176
top5_accuracy_validation_degraded["Pretrained ResNet50 (ImageNet1k) Augmented Training Pytorch Weights"] =  35.55776892430279
accuracy_validation["Pretrained ResNet50 (ImageNet1k) Augmented Training Trained Weights"] = 26.195219123505975
top5_accuracy_validation["Pretrained ResNet50 (ImageNet1k) Augmented Training Trained Weights"] = 52.788844621513945
accuracy_validation_degraded["Pretrained ResNet50 (ImageNet1k) Augmented Training Trained Weights"] = 18.92430278884462
top5_accuracy_test_degraded["Pretrained ResNet50 (ImageNet1k) Augmented Training Trained Weights"] =  36.05577689243028
accuracy_validation["Pretrained ResNet50 (FOOD101) Augmented Training Original Weights"] = 28.984063745019924
top5_accuracy_validation["Pretrained ResNet50 (FOOD101) Augmented Training Original Weights"] =  54.482071713147405
accuracy_validation_degraded["Pretrained ResNet50 (FOOD101) Augmented Training Original Weights"] = 24.40239043824701
top5_accuracy_validation_degraded["Pretrained ResNet50 (FOOD101) Augmented Training Original Weights"] =  47.808764940239044
accuracy_validation["Pretrained ResNet50 (FOOD101) Augmented Training Trained Weights"] = 28.286852589641438
top5_accuracy_validation["Pretrained ResNet50 (FOOD101) Augmented Training Trained Weights"] =  51.79282868525896
accuracy_validation_degraded["Pretrained ResNet50 (FOOD101) Augmented Training Trained Weights"] = 23.107569721115535
top5_accuracy_validation_degraded["Pretrained ResNet50 (FOOD101) Augmented Training Trained Weights"] =  45.41832669322709
accuracy_test["Pretrained ResNet50 (FOOD101)"] = 39.73653493413373
top5_accuracy_test["Pretrained ResNet50 (FOOD101)"] = 67.60046690011673
accuracy_test_degraded["Pretrained ResNet50 (FOOD101)"] = 26.721694180423544
top5_accuracy_test_degraded["Pretrained ResNet50 (FOOD101)"] =  48.79939969984993
accuracy_test["Pretrained ResNet50 (ImageNet1k)"] = 29.87326996831749
top5_accuracy_test["Pretrained ResNet50 (ImageNet1k)"] =  55.61113890278473
accuracy_test_degraded["Pretrained ResNet50 (ImageNet1k)"] = 18.325829581457395
top5_accuracy_test_degraded["Pretrained ResNet50 (ImageNet1k)"] =  36.87677171919293
accuracy_test["Pretrained ResNet50 (FOOD101) Augmented Training Original Weights"] = 27.69418042354049
top5_accuracy_test["Pretrained ResNet50 (FOOD101) Augmented Training Original Weights"] =  62.673003168250794
accuracy_test_degraded["Pretrained ResNet50 (FOOD101) Augmented Training Original Weights"] = 29.214607303651825
top5_accuracy_test_degraded["Pretrained ResNet50 (FOOD101) Augmented Training Original Weights"] =  54.98582624645656


In [6]:
# Convert dictionaries to Pandas Series
import pandas as pd

dicts = {
    "accuracy_validation": accuracy_validation,
    "top5_accuracy_validation": top5_accuracy_validation,
    "accuracy_test": accuracy_test,
    "top5_accuracy_test": top5_accuracy_test,
    "accuracy_validation_degraded": accuracy_validation_degraded,
    "top5_accuracy_validation_degraded": top5_accuracy_validation_degraded,
    "accuracy_test_degraded": accuracy_test_degraded,
    "top5_accuracy_test_degraded": top5_accuracy_test_degraded
}

# Create a DataFrame by concatenating the Series
df = pd.DataFrame({name: pd.Series(data) for name, data in dicts.items()})

# Fill NaN values with a placeholder (optional)
df.fillna("N/A", inplace=True)
df.sort_values(by="accuracy_validation", ascending=True, inplace=True)

# Display DataFrame
df.head(20)

C:\Users\marco\AppData\Local\Temp\ipykernel_12372\310768856.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("N/A", inplace=True)


,accuracy_validation,top5_accuracy_validation,accuracy_test,top5_accuracy_test,accuracy_validation_degraded,top5_accuracy_validation_degraded,accuracy_test_degraded,top5_accuracy_test_degraded
Classify To Learn ResNet50,0.298805,1.693227,N/A,N/A,N/A,N/A,N/A,N/A
Colorizer + Labels Clustering,0.398406,2.490040,N/A,N/A,N/A,N/A,N/A,N/A
Colorizer + Data Augmentation,0.498008,2.788845,N/A,N/A,N/A,N/A,N/A,N/A
Sparse Autoencoder,0.796813,2.091633,N/A,N/A,N/A,N/A,N/A,N/A
Colorizer,1.095618,3.386454,N/A,N/A,N/A,N/A,N/A,N/A
Baseline,1.593625,6.175299,N/A,N/A,N/A,N/A,N/A,N/A
Pretrained ResNet50 (ImageNet1k) Augmented Training Pytorch Weights,22.808765,46.314741,N/A,N/A,16.23506,35.557769,N/A,N/A
Pretrained ResNet50 (ImageNet1k),25.398406,49.402390,29.87327,55.611139,N/A,N/A,18.32583,36.876772
Pretrained ResNet50 (ImageNet1k) Augmented Training Trained Weights,26.195219,52.788845,N/A,N/A,18.924303,N/A,N/A,36.055777
Pretrained ResNet50 (FOOD101) Augmented Training Trained Weights,28.286853,51.792829,N/A,N/A,23.10757,45.418327,N/A,N/A


In [14]:
df.to_csv("misc/model_metrics/metrics.csv")